In [ ]:
!pip install timm torchvision
!pip install wandb
!pip install torch_optimizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
from torch_optimizer import Ranger
from torchvision import transforms
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader, random_split
import timm
from timm.loss import LabelSmoothingCrossEntropy
from timm.models.layers import DropPath
import data_preprocessing
from wandb_logger import WandBLogger
from torch.cuda.amp import autocast, GradScaler
import os
import random
import numpy as np
from datetime import datetime

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:
SEED = 42

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)

In [ ]:
def build_optimizer(optimizer_name, model):
    if optimizer_name.lower() == "adamw":
        config = {
            "learning_rate": LR,
            "betas": BETAS,
            "weight_decay": WEIGHT_DECAY
        }
        optimizer = optim.AdamW(
            model.parameters(),
            lr=LR,
            betas=BETAS,
            weight_decay=WEIGHT_DECAY
        )
    elif optimizer_name.lower() == "sgd":
        config = {
            "learning_rate": LR,
            "momentum": MOMENTUM,
            "weight_decay": WEIGHT_DECAY,
            "nesterov": NESTEROV
        }
        optimizer = optim.SGD(
            model.parameters(),
            lr=LR,
            momentum=MOMENTUM,
            weight_decay=WEIGHT_DECAY,
            nesterov=False
        )
    elif optimizer_name.lower() == "ranger":
        config = {
            "learning_rate": LR,
            "betas": BETAS,
            "weight_decay": WEIGHT_DECAY
        }
        # Ranger optimizer setup
        optimizer = Ranger(
            model.parameters(),
            lr=LR,
            betas=BETAS,
            weight_decay=WEIGHT_DECAY
        )
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}")

    print(f"Optimizer '{optimizer_name}' initialized successfully.")
    return config, optimizer

def build_scheduler(scheduler_name, optimizer):
    if scheduler_name.lower() == "ranger + onecyclelr":
        config = {
            "max_lr": LR,
            "epochs": EPOCHS,
            "steps_per_epoch": len(trainloader),
            "pct_start": 0.3,
            "anneal_strategy": "cos",
            "div_factor": 25.0,
            "final_div_factor": 1e4
        }
        scheduler = OneCycleLR(
            optimizer,
            max_lr=LR,
            epochs=EPOCHS,
            steps_per_epoch=len(trainloader),
            pct_start=0.3,
            anneal_strategy="cos",
            div_factor=25.0,
            final_div_factor=1e4
        )
    elif scheduler_name.lower() == "cosineannealinglr + warmup":
        config = {
            "warmup_epochs": WARMUP_EPOCHS,
            "start_factor": START_FACTOR,
            "t_max": T_MAX,
            "milestones": [WARMUP_EPOCHS]
        }
        scheduler = optim.lr_scheduler.SequentialLR(
            optimizer,
            schedulers=[
                optim.lr_scheduler.LinearLR(optimizer, start_factor=START_FACTOR, total_iters=WARMUP_EPOCHS),
                optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_MAX)
            ],
            milestones=[WARMUP_EPOCHS]
        )
    elif scheduler_name.lower() == "cosineannealingwarmrestarts":
        config = {
            "T_0": 10,
            "T_mult": 2
        }
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer,
            T_0=10,
            T_mult=2
        )
    else:
        raise ValueError(f"Unsupported scheduler: {scheduler_name}")

    print(f"Scheduler '{scheduler_name}' initialized successfully.")
    return config, scheduler

In [ ]:
!wandb login 8d8bd8c6f482323212f368b2070800726beaea94

wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [ ]:
# THE WHOLE TRAINING SETTINGS ARE HERE!

# GENERAL
BATCH_SIZE = 128
EPOCHS = 30
VAL_SPLIT = 0.1
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BACKBONE_FREEZING = True

# LOSS
SMOOTHING=0.1
# OPTIMIZER
OPTIMIZER_NAME = "AdamW"
LR=0.001
BETAS=(0.9, 0.999)
WEIGHT_DECAY=0.05
WARMUP_EPOCHS=5
NESTEROV=False
MOMENTUM=0.9
# SCHEDULER
SCHEDULER_NAME = "CosineAnnealingWarmRestarts"
START_FACTOR=1e-6 / 5e-5
T_MAX=EPOCHS - WARMUP_EPOCHS
# Early Stopping
PATIENCE = 6

In [ ]:
pipeline = data_preprocessing.CIFAR100Pipeline(val_split=VAL_SPLIT, use_augment=True)
trainset, valset, testset = pipeline.run_pipeline()
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
valloader = DataLoader(valset, batch_size=BATCH_SIZE)
testloader = DataLoader(testset, batch_size=BATCH_SIZE)

100%|██████████| 169M/169M [00:03<00:00, 43.2MB/s]


In [ ]:
def create_dino_vit_s16_for_cifar100(freezing=False):
    model = timm.create_model("vit_small_patch16_224_dino", pretrained=True, num_classes=0, drop_path_rate=0.1)

    # Replace the head with CIFAR-100 classification head
    model.head = nn.Linear(model.num_features, 100)

    if freezing:
      # Freeze all parameters except head
      for param in model.parameters():
          param.requires_grad = False

      # Unfreeze only the head
      for param in model.head.parameters():
          param.requires_grad = True

    return model

model = create_dino_vit_s16_for_cifar100(BACKBONE_FREEZING).to(DEVICE)

OPTIMIZER_CONFIG, optimizer = build_optimizer(OPTIMIZER_NAME, model)
SCHEDULER_CONFIG, scheduler = build_scheduler(SCHEDULER_NAME, optimizer)

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name vit_small_patch16_224_dino to current vit_small_patch16_224.dino.
  model = create_fn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

Optimizer 'AdamW' initialized successfully.
Scheduler 'CosineAnnealingWarmRestarts' initialized successfully.


In [ ]:
print(next(model.parameters()).device)
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable:,} / {total:,}")

torch.backends.cudnn.benchmark = True

cuda:0
Trainable params: 38,500 / 21,704,164


In [ ]:
run_name = f"CENTRALIZED_{datetime.now().strftime('%Y%m%d-%H%M%S')}"

logger = WandBLogger(
    project_name="federated-learning-project",
    run_name=run_name,
    config={
        # Training Setup
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "device": DEVICE,
        "backbone_freezing": BACKBONE_FREEZING,

        # Loss Configuration
        "label_smoothing": SMOOTHING,

        # Optimizer Configuration
        "optimizer": OPTIMIZER_NAME,
        "optimizer_config": OPTIMIZER_CONFIG,

        # Scheduler Configuration
        "scheduler": SCHEDULER_NAME,
        "scheduler_config": SCHEDULER_CONFIG,

        # Early Stopping
        "patience": PATIENCE
    }
)


wandb: Currently logged in as: sotoudehfarnood (polito-fl) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Loss with label smoothing
criterion = LabelSmoothingCrossEntropy(smoothing=SMOOTHING)

scaler = GradScaler()

best_val_acc = 0.0
epochs_no_improve = 0

# Training loop
for epoch in range(EPOCHS):
    model.train()
    correct, total, train_loss = 0, 0, 0.0

    for x, y in trainloader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()

        with autocast():
            outputs = model(x)
            loss = criterion(outputs, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # scheduler.step()

        train_loss += loss.item() * y.size(0)
        _, pred = torch.max(outputs, 1)
        correct += (pred == y).sum().item()
        total += y.size(0)

    scheduler.step()

    train_acc = correct / total
    train_loss /= total

    logger.log_metrics({
      "train_loss": train_loss,
      "train_acc": train_acc,
      "learning_rate": scheduler.get_last_lr()[0]
    }, step=epoch)


    # Validation
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    with torch.no_grad():
        for x, y in valloader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = model(x)
            loss = criterion(outputs, y)

            val_loss += loss.item() * y.size(0)
            _, pred = torch.max(outputs, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)

    val_acc = correct / total
    val_loss /= total
    logger.log_metrics({
      "val_loss": val_loss,
      "val_acc": val_acc
    }, step=epoch)

    print(f"Epoch {epoch+1:02d}/{EPOCHS} — Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # Early stopping logic
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        epochs_no_improve = 0
        best_model_state = model.state_dict()  # save best model
        logger.log_model(model, path="best_model.pth")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

model.load_state_dict(best_model_state)

<ipython-input-12-146f11194714>:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-12-146f11194714>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 01/30 — Train Acc: 0.5376 | Val Acc: 0.6824
Epoch 02/30 — Train Acc: 0.6915 | Val Acc: 0.7210
Epoch 03/30 — Train Acc: 0.7244 | Val Acc: 0.7280
Epoch 04/30 — Train Acc: 0.7395 | Val Acc: 0.7392
Epoch 05/30 — Train Acc: 0.7586 | Val Acc: 0.7546
Epoch 06/30 — Train Acc: 0.7711 | Val Acc: 0.7540
Epoch 07/30 — Train Acc: 0.7881 | Val Acc: 0.7584
Epoch 08/30 — Train Acc: 0.7995 | Val Acc: 0.7662
Epoch 09/30 — Train Acc: 0.8106 | Val Acc: 0.7698
Epoch 10/30 — Train Acc: 0.8176 | Val Acc: 0.7720
Epoch 11/30 — Train Acc: 0.7394 | Val Acc: 0.7378
Epoch 12/30 — Train Acc: 0.7423 | Val Acc: 0.7308
Epoch 13/30 — Train Acc: 0.7480 | Val Acc: 0.7348
Epoch 14/30 — Train Acc: 0.7540 | Val Acc: 0.7354
Epoch 15/30 — Train Acc: 0.7564 | Val Acc: 0.7320
Epoch 16/30 — Train Acc: 0.7616 | Val Acc: 0.7388
Early stopping triggered at epoch 16


<All keys matched successfully>

In [ ]:
#  test
model.eval()
correct, total, test_loss = 0, 0, 0.0
with torch.no_grad():
    for x, y in testloader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        outputs = model(x)
        loss = criterion(outputs, y)

        test_loss += loss.item() * y.size(0)
        _, pred = torch.max(outputs, 1)
        correct += (pred == y).sum().item()
        total += y.size(0)

test_acc = correct / total
test_loss /= total

logger.log_metrics({
    "test_loss": test_loss,
    "test_acc": test_acc
})

logger.finish()

print(f"\n Final Test Accuracy: {test_acc:.4f} | Test Loss: {test_loss:.4f}")


learning_rate,█▇▇▆▄▃▂▂▁████▇▇▇
test_acc,▁
test_loss,▁
train_acc,▁▅▆▆▇▇▇███▆▆▆▆▆▇
train_loss,█▄▃▂▂▂▁▁▁▁▂▂▂▂▂▂
val_acc,▁▄▅▅▇▇▇███▅▅▅▅▅▅
val_loss,█▅▄▄▃▂▂▁▁▁▄▅▄▄▄▄
learning_rate,0.00079
test_acc,0.7425
test_loss,1.73375
train_acc,0.7616



 Final Test Accuracy: 0.7425 | Test Loss: 1.7337
